# Process events files and generates trials for sessions after to 20240416

In [1]:
import os
import shutil

import session_processing_helper_c5v2 as helper
import utils

import pandas as pd

In [2]:
data_dir = '/Users/rebekahzhang/data/behavior_data'
period = 'post_meta_change'
exp = "exp2"
cohort = "cohort_8"
data_folder = os.path.join(data_dir, period, exp, cohort)
print(data_folder)

/Users/rebekahzhang/data/behavior_data/post_meta_change/exp2/cohort_8


# Quality Control

## Check session folders have both meta and events

In [3]:
missing_events, missing_meta, empty_meta, empty_events = helper.check_session_files(data_folder)
if not (missing_meta.empty and missing_events.empty and empty_meta.empty and empty_events.empty):
    print("\nFile check results:")
    if not missing_meta.empty:
        print("\nSessions missing meta files:")
        display(missing_meta)
    if not missing_events.empty:
        print("\nSessions missing events files:")
        display(missing_events)
    if not empty_meta.empty:
        print("\nSessions with empty meta files:")
        display(empty_meta)
    if not empty_events.empty:
        print("\nSessions with empty events files:")
        display(empty_events)
else:
    print("\nAll sessions have non-empty meta and events files.")


All sessions have non-empty meta and events files.


In [4]:
utils.delete_folders(missing_meta.dir.tolist(), data_folder)

no sessions to delete


In [5]:
utils.delete_folders(missing_events.dir.tolist(), data_folder)

no sessions to delete


In [6]:
utils.delete_folders(empty_meta.dir.tolist(), data_folder)

no sessions to delete


In [7]:
utils.delete_folders(empty_events.dir.tolist(), data_folder)

no sessions to delete


## Generate and save sessions log

generate session log using meta data from each session and add columns of basic info to each session

### re-run after every quality control steps

In [8]:
sessions_all, sessions_training = helper.generate_session_logs(data_folder)
sessions_training.tail()

36 sessions in total


,index,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group,session
31,12,2025-08-19,12-40-10,RZ075,2,regular,rig3,Rebekah,True,700,712,0.94,reward,2025-08-19_12-40-10_RZ075,l,9
32,7,2025-08-20,10-30-09,RZ074,2,regular,rig3,Rebekah,True,700,256,7.40,reward,2025-08-20_10-30-09_RZ074,l,11
33,35,2025-08-20,10-41-37,RZ077,2,regular,rig6,Rebekah,False,700,359,2.49,reward,2025-08-20_10-41-37_RZ077,l,6
34,11,2025-08-20,10-44-55,RZ078,2,regular,rig7,Rebekah,False,700,380,1.97,reward,2025-08-20_10-44-55_RZ078,l,5
35,33,2025-08-20,11-56-35,RZ075,2,regular,rig3,Rebekah,True,565,294,8.44,miss,2025-08-20_11-56-35_RZ075,l,10


## Remove unwanted sessions
doesn't need to run when data folder is cleaned 
<br>
sessions_all needs to be regenerated after every cleaning step

### Remove test sessions
currently not needed cuz test sessions are not sorted into exp folders

In [9]:
sessions_test = sessions_all.loc[sessions_all.mouse=='test']
utils.remove_sessions(sessions_test, data_folder)

no sessions to delete


### Remove crashed sessions
remove the sessions with ending_code==nan

In [10]:
sessions_ended_nan = sessions_all[sessions_all.ending_code.isna()]
utils.remove_sessions(sessions_ended_nan, data_folder)

no sessions to delete


reomve sessions that crashed

In [11]:
sessions_crashed = pd.DataFrame(columns=sessions_training.columns)
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)

for _, session_info in sessions_training.iterrows():
    try:
        events_path = utils.generate_events_path(data_folder, session_info)
        events = pd.read_csv(events_path, low_memory=False)
        session_end = events.loc[(events.key=='session') & (events.value==0)]

        if not len(session_end)==1:
            sessions_crashed = pd.concat([sessions_crashed, session_info.to_frame().T], ignore_index=True)
    
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

if len(problematic_sessions) > 0:
    print("cannot open: ")
    display(problematic_sessions)

if len(sessions_crashed) > 0:
    print("crashed sessions: ")
    display(sessions_crashed)
else:
    print("all sessions are perfect! woohoo!")

all sessions are perfect! woohoo!


In [12]:
utils.remove_sessions(sessions_crashed, data_folder)

no sessions to delete


### Check for short sessions

In [13]:
short_threshold = 20
sessions_short = sessions_all[(sessions_all['total_trial'] < short_threshold) | sessions_all['total_trial'].isna()]
if len(sessions_short)>0:
    display(sessions_short)
else: 
    print('no short sessions to be checked!')

no short sessions to be checked!


remove short sessions if needed

In [14]:
utils.remove_sessions(sessions_short, data_folder)

no sessions to delete


# Process Events

load session log

In [15]:
sessions_training = utils.load_data(os.path.join(data_folder, 'sessions_training.csv'))

In [16]:
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)

for _, session_info in sessions_training.iterrows():
    try:
        events_processed_path = utils.generate_events_processed_path(data_folder, session_info)
        if os.path.isfile(events_processed_path):
            continue
        events = pd.read_csv(utils.generate_events_path(data_folder, session_info), low_memory=False)
        events = helper.process_events(session_info, events)
        events_processed = events.groupby('session_trial_num', group_keys=False).apply(helper.add_trial_time)
        events_processed.to_csv(events_processed_path)
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

if len(problematic_sessions) > 0:
    display(problematic_sessions)
else:
    print("all sessions are perfect! woohoo!")

all sessions are perfect! woohoo!


# Data set curation 

In [19]:
mouse_list = utils.generate_mouse_list(sessions_all)
# mouse_list = ['RZ047','RZ049','RZ050','RZ051','RZ052','RZ053','RZ054','RZ055','RZ056']
# mouse_list = ["RZ057", "RZ058", "RZ059", "RZ061", "RZ062", "RZ063",
#                      "RZ065", "RZ067", "RZ068", "RZ069", "RZ070"]
print(mouse_list)

sessions_by_date = sessions_training.groupby('date')

['RZ074', 'RZ075', 'RZ077', 'RZ078']


### Deal with missing sessions
this is not the proper way to deal with this. should have it populated with mean and variation

In [20]:
no_missing_sessions = True
for date, data in sessions_by_date:
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) < 1:
            no_missing_sessions = False
            print(f"on {date}, {mouse} has missing sessions")
if no_missing_sessions:
    print("no missing sessions!")

on 2025-07-31, RZ075 has missing sessions
on 2025-07-31, RZ077 has missing sessions
on 2025-07-31, RZ078 has missing sessions
on 2025-08-04, RZ077 has missing sessions
on 2025-08-04, RZ078 has missing sessions
on 2025-08-05, RZ077 has missing sessions
on 2025-08-05, RZ078 has missing sessions
on 2025-08-06, RZ077 has missing sessions
on 2025-08-06, RZ078 has missing sessions
on 2025-08-07, RZ077 has missing sessions
on 2025-08-07, RZ078 has missing sessions
on 2025-08-08, RZ078 has missing sessions


In [24]:
sessions_by_date.get_group('2025-08-04').sort_values('mouse')

,index,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group,session
1,22,2025-08-04,13-25-36,RZ074,2,regular,rig2,Rebekah,False,700,269,8.31,reward,2025-08-04_13-25-36_RZ074,l,1
2,1,2025-08-04,13-30-07,RZ075,2,regular,rig3,Rebekah,False,275,138,2.62,time,2025-08-04_13-30-07_RZ075,l,0


#### duplicate if you are sussed out of having to redo this

In [25]:
utils.backup(data_folder)

cohort_8_copy already exist


### Deal with multiple sessions

In [26]:
days_to_stitch = []
mice_to_stitch = []
for date, data in sessions_by_date:
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) > 1:
            days_to_stitch.append(date)
            mice_to_stitch.append(mouse)
            print(f"on {date}, {mouse} has {len(mouse_by_date)} sessions")
if not days_to_stitch:
    print("no sessions to stitch!")

no sessions to stitch!


In [27]:
# run it if session stitching is needed, nothing would happen otherwise
# has to run more than once if there are more than 2 sessions. fix it for the next round pls
if not days_to_stitch:
    print("no sessions to stitch!")
else:
    for d, m in zip(days_to_stitch, mice_to_stitch):
        day = sessions_by_date.get_group(d)
        sessions_to_stitch = day[day['mouse'] == m]

        session_1_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[0])
        session_2_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[1])

        if os.path.exists(session_1_dir) and os.path.exists(session_2_dir):
            session_1 = pd.read_csv(session_1_dir)
            session_2 = pd.read_csv(session_2_dir)
            stitched_session = helper.stitch_sessions(session_1, session_2) 
            #should change to stitch events. stitch sessions should be deleted. to follow nomanclature, all session should be renamed to events!!

            stitched_session.to_csv(session_1_dir, index=False)
            shutil.rmtree(os.path.join(data_folder, sessions_to_stitch.iloc[1].dir))
            print(f"{d} {m} session 2 deleted")
        else:
            print("one of the sessions do not exist")

no sessions to stitch!


In [28]:
sessions_all, sessions_training = helper.generate_session_logs(data_folder)

sessions_training.tail()

36 sessions in total


,index,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group,session
31,12,2025-08-19,12-40-10,RZ075,2,regular,rig3,Rebekah,True,700,712,0.94,reward,2025-08-19_12-40-10_RZ075,l,9
32,7,2025-08-20,10-30-09,RZ074,2,regular,rig3,Rebekah,True,700,256,7.40,reward,2025-08-20_10-30-09_RZ074,l,11
33,35,2025-08-20,10-41-37,RZ077,2,regular,rig6,Rebekah,False,700,359,2.49,reward,2025-08-20_10-41-37_RZ077,l,6
34,11,2025-08-20,10-44-55,RZ078,2,regular,rig7,Rebekah,False,700,380,1.97,reward,2025-08-20_10-44-55_RZ078,l,5
35,33,2025-08-20,11-56-35,RZ075,2,regular,rig3,Rebekah,True,565,294,8.44,miss,2025-08-20_11-56-35_RZ075,l,10


# Finalize sessions log

In [29]:
def correct_sessions_training(data_folder, save_log=True):
    _, sessions_training = helper.generate_session_logs(data_folder, save_logs=False)
    session_info_list = []
    for _, session_info in sessions_training.iterrows():
        events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info), low_memory=False)
        session_basics = helper.get_session_basics(events_processed)
        session_basics['dir'] = session_info['dir']
        session_info_list.append(session_basics)
    sessions_info = pd.DataFrame(session_info_list)
    corrected_sessions_training = pd.merge(sessions_training, sessions_info, on="dir")
    corrected_sessions_training = corrected_sessions_training.drop(columns=['index', 'total_reward', 'total_trial', 'total_reward'])
    corrected_sessions_training = corrected_sessions_training.groupby('mouse', group_keys=False).apply(helper.assign_session_numbers)
    corrected_sessions_training['cohort'] = cohort
    if save_log:
        utils.save_as_csv(df=corrected_sessions_training, folder=data_folder, filename=f'sessions_training_{exp}_{cohort}.csv')
    return corrected_sessions_training

In [30]:
sessions_training = correct_sessions_training(data_folder)

36 sessions in total


# Analyze events

In [31]:
sessions_training = utils.load_data(os.path.join(data_folder, f'sessions_training_{exp}_{cohort}.csv'))

## Generate Trials

In [32]:
# generate all trials based on events processed
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)
for _, session_info in sessions_training.iterrows():
    try: 
        trials_path = utils.generate_trials_path(data_folder, session_info)
        if os.path.isfile(trials_path):
            continue
        
        events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info))
        trials = helper.generate_trials(session_info, events_processed)

        trials.to_csv(trials_path)
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

if len(problematic_sessions) > 0:
    display(problematic_sessions)
else:
    print("all sessions are perfect! woohoo!")

all sessions are perfect! woohoo!


## Analyze trials

In [33]:
for _, session_info in sessions_training.iterrows():
    try:
        trials_analyzed_path = utils.generate_trials_analyzed_path(data_folder, session_info)
        if os.path.isfile(trials_analyzed_path):
            continue
        
        session_by_trial = utils.load_data(utils.generate_events_processed_path(data_folder, session_info)).groupby('session_trial_num')
        trials = utils.load_data(utils.generate_trials_path(data_folder, session_info))
        trials_data = helper.get_trial_data_df(session_by_trial)
        trials_analyzed = pd.merge(trials, trials_data, on='session_trial_num')
        trials_analyzed['group'] = session_info.group #assigning trial type manually
        trials_analyzed.to_csv(trials_analyzed_path)
    except:
        display(session_info)

debug debug debug

In [ ]:
# data_dir = '/Users/rebekahzhang/data/behavior_data'
# exp = "exp2"
# cohort = "c567"
# data_folder = os.path.join(data_dir, exp, cohort)
# sessions_training = pd.read_csv(os.path.join(data_folder, 'sessions_training.csv'), index_col=0)

# sessions_training_post_meta_change = sessions_training.loc[sessions_training.date < "2024-05-03"]

# for _, session_info in sessions_training_post_meta_change.iterrows():
#     try:
#         trials_analyzed_path = utils.generate_trials_analyzed_path(data_folder, session_info)
#         # if os.path.isfile(trials_analyzed_path):
#         #     continue
        
#         session_by_trial = utils.load_data(utils.generate_events_processed_path(data_folder, session_info)).groupby('session_trial_num')
#         trials = utils.load_data(utils.generate_trials_path(data_folder, session_info))
#         trials_data = helper.get_trial_data_df(session_by_trial)
#         trials_analyzed = pd.merge(trials, trials_data, on='session_trial_num')
#         trials_analyzed['group'] = session_info.group #assigning trial type manually
#         trials_analyzed.to_csv(trials_analyzed_path)
#     except:
#         display(session_info)